# Projet Sport Metrics – Notebook de pipeline

## 1. Contexte et objectifs

Ce notebook documente le pipeline de données du projet **Sport Metrics**, qui vise à relier données d’entraînement, biométrie et performances de match pour analyser la fatigue, le risque de blessure et la performance collective.  
Le pipeline va des fichiers sources (Google Drive, API NBA) jusqu’aux tables analytiques dans BigQuery, en passant par GitHub, Airbyte, Airflow et dbt.

## 2. Vue d’ensemble du pipeline

Étapes principales :

1. Extraction des fichiers CSV depuis Google Drive vers GitHub (versionning) via Airflow (Python).
2. Ingestion des CSV GitHub dans Airbyte, puis chargement dans BigQuery (zone brute).
3. Nettoyage et modélisation des tables brutes avec dbt dans un schéma analytique dédié.
4. Exploitation des tables/vues finales dans BigQuery pour les analyses “fatigue”, “blessures” et “performance”.

## 3. Modèle de données cible

### 3.1 Table `exercise_type`

Table de référence des types d’exercices.

- PK : `exercise_id` (SERIAL)
- Colonnes principales :
  - `exercise_name` (VARCHAR) : libellé de l’exercice

### 3.2 Table `players_infos`

Informations personnelles et biométriques des joueurs.

- PK : `player_id` (SERIAL)
- Identité : `first_name`, `last_name`, `nickname`, `birthdate`, `school`, `country`
- Biométrie : `height_cm`, `weight_kg`
- Parcours : `season_exp`, `jersey_number`, `first_position`, `second_position`, `start_year`, `end_year`, `draft_year`, `age`

### 3.3 Table `players_stats`

Statistiques de match au niveau joueur.

- PK : `player_stats_id` (SERIAL)
- FK : `game_id`, `team_id`, `player_id`
- Variables (exemples) :
  - Temps de jeu et rôle : `min_played`, `start_position`, `availability_comment`
  - Performance : `pts`, `plus_minus`, autres métriques de stats de match (rebonds, passes, etc.)

### 3.4 Table `teams`

Référentiel des équipes.

- PK : `team_id`
- Colonnes : `team_name`, `team_code`, `team_town`

### 3.5 Table `training_sessions`

Données d’entraînement par session et par joueur.

- PK : `session_id`
- FK : `team_code`, `next_match_id` (référence `game_id`), `player_id`, `exercise_id`
- Colonnes (exemples) :
  - Contexte : date/heure de session, lien avec match (`next_match_id`)
  - Joueur : `player_id`, âge, position
  - Biométrie dynamique / temporelle : variables de charge et de fatigue
  - Performance à l’entraînement

### 3.6 Tables de matchs et boxscores (`games_boxscore`, `team_boxscores`, `team_games_dataset`)

- `games_boxscore` : match global (HOME vs AWAY) avec `game_id`, `game_date`, `team_id_home`, `team_id_away`, stats globales, `win_loss`, `min_played`, `pts`, `plus_minus`
- `team_boxscores` : stats par équipe pour chaque match, avec `game_id`, `team_id` et stats agrégées
- `team_games_dataset` (vue / table analytique dbt) :
  - PK : `game_id`
  - FK : `team_id` (via `team_code`)
  - Variables : `game_date`, `matchup` (HOME vs AWAY), `win_loss`, `min_played`, `pts` et autres statistiques de performance

## 4. Étapes d’ingestion (Drive → GitHub → Airbyte → BigQuery brut)

### 4.1 Google Drive → GitHub (Airflow, Python)

Objectif : versionner les fichiers CSV sources dans GitHub.

- DAG Airflow `drive_to_github_dag` :
  - Tâche 1 : se connecter à Google Drive et télécharger les CSV (par ex. `training_sessions.csv`)
  - Tâche 2 : copier les fichiers dans le repo local, faire `git add / commit / push` vers GitHub

### 4.2 GitHub → Airbyte

Objectif : déclarer GitHub comme source de données dans Airbyte.

- Connexion Airbyte “GitHub CSV → BigQuery RAW”
- Airflow déclenche la sync Airbyte (API) pour charger les fichiers CSV vers BigQuery (schéma brut)

### 4.3 Airbyte → BigQuery (zone BRUT)

Résultat : tables brutes dans BigQuery, par exemple :

- `raw_training_sessions`
- `raw_players_infos`
- `raw_players_stats`
- `raw_games_boxscore`
- `raw_team_boxscores`

## 5. Étapes de transformation (dbt → BigQuery ANALYSE)

### 5.1 Cleaning & normalisation

dbt lit les tables `raw_*` dans BigQuery et applique :

- Harmonisation des noms de colonnes et formats de dates
- Définition des PK/FK de référence (`player_id`, `game_id`, `team_id`, `exercise_id`)
- Création de tables dimension (`players_infos`, `teams`, `exercise_type`) et de tables de faits (`players_stats`, `training_sessions`, `team_games_dataset`)

### 5.2 Modèles analytiques dbt

Exemples de modèles / vues finales :

- Modèle `dim_players_infos` : vue propre sur la table `players_infos` (colonnes nettoyées, éventuellement enrichies)
- Modèle `fct_players_stats` : stats de match joueur avec clés `player_id`, `game_id`, `team_id`, métriques de performance
- Modèle `fct_training_sessions` : sessions d’entraînement reliées aux matchs (`next_match_id` → `game_id`) et à `exercise_type`
- Modèle `fct_team_games` / `team_games_dataset` : stats de match par équipe (résultat, points, minutes, indicateurs d’ADN de la victoire)

## 6. Axes d’analyse métier

### 6.1 Fatigue et load management

À partir de `fct_training_sessions` et `fct_players_stats` :

- Moyenne du temps de récupération par session et par joueur / équipe
- `fatigue_level`, `load_intensity_score`, `duration_min` par session et par joueur
- Lien avec la performance en match (`pts`, `plus_minus`, `start_position`)

### 6.2 Prévention des blessures

En croisant biométrie (`players_infos`), charge d’entraînement (`fct_training_sessions`) et éventuel `injury_risk_level` :

- Profil de risque par joueur (âge, taille, poids, historique de charge)
- Identification de patterns de sessions “à risque” (combinaisons d’exercices, intensité, temps de récupération)

### 6.3 Performance collective et stratégie

À partir de `team_games_dataset` et `fct_players_stats` :

- ADN de la victoire : impact de `reb`, `tov`, `fg_pct`, `ast`, `blk` sur la probabilité de victoire
- Domicile vs extérieur : taux de victoire par `matchup` (HOME vs AWAY)
- Efficacité minute par minute : ratio `pts / min_played` par joueur / ligne